In [140]:
from sklearn.datasets import load_iris
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import pandas as pd;
import numpy as np

In [141]:
city_name = {"Albury":1,"BadgreyCreek":2,"Cobar":3,"CoffsHarbour":4,"Moree":5,"Newcastle":6}
day_per_mouth = [31,28,31,30,31,30,31,31,30,31,30,31]
pro = {"Yes":1,"No":0}

In [142]:
data = pd.read_csv("weather.csv")
data = data.drop("Evaporation",axis=1)
data = data.drop("Sunshine",axis=1)

In [143]:
from datetime import date
def updating_data_column(column):
    for i in range(len(column)):
        temp = column[i].split('-')
        temp[0] = temp[0][2:4]
        column[i] = int(''.join(temp))
    return column;

In [144]:
def transforming(column,values):
  index = 0
  for i in column:
    if(i not in values):
      index += 1;
      values.update({i:index})
  return values;

In [145]:
column = data['WindDir3pm']
column2 = data['WindDir9am']
column3 = data['WindGustDir']
values = transforming(column,{})
values = transforming(column2,values)
values = transforming(column3,values)
info = {}
for i in values:
  if(values[i] != 16):
    info.setdefault(i,values[i])

In [146]:
data['WindDir3pm'] = data['WindDir3pm'].map(values)
data['WindDir9am'] = data['WindDir9am'].map(values)
data['WindGustDir'] = data['WindGustDir'].map(values)
data['Location'] = data['Location'].map(city_name)
data['RainToday'] = data['RainToday'].map(pro)
data['RainTomorrow'] = data['RainTomorrow'].map(pro)
data['Date'] = updating_data_column(data['Date'])

<ipython-input-143-fbe1861e946e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column[i] = int(''.join(temp))


In [147]:
def filling_out(df):
  for col in df.columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    # Identify outliers based on the IQR
    outliers = df[(df[col] < Q1 - 1.5 * IQR) | (df[col] > Q3 + 1.5 * IQR)]

    # Calculate the mean of the column, excluding the outliers
    mean = df[~df[col].isin(outliers[col])][col].mean()

    # Replace the outliers with the mean value
    df.loc[outliers.index,col] = mean

    # Remove the outliers from the DataFrame
    df = df[~df[col].isin(outliers[col])]
  return df

In [148]:
data = data.iloc[:, 1:]
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           25000 non-null  object 
 1   Location       14761 non-null  float64
 2   MinTemp        24669 non-null  float64
 3   MaxTemp        24824 non-null  float64
 4   Rainfall       24721 non-null  float64
 5   WindGustDir    25000 non-null  int64  
 6   WindGustSpeed  21545 non-null  float64
 7   WindDir9am     25000 non-null  int64  
 8   WindDir3pm     25000 non-null  int64  
 9   WindSpeed9am   24428 non-null  float64
 10  WindSpeed3pm   23770 non-null  float64
 11  Humidity9am    24609 non-null  float64
 12  Humidity3pm    23936 non-null  float64
 13  Pressure9am    20172 non-null  float64
 14  Pressure3pm    20173 non-null  float64
 15  Cloud9am       14136 non-null  float64
 16  Cloud3pm       13815 non-null  float64
 17  Temp9am        24755 non-null  float64
 18  Temp3p

In [149]:
i = 0;
j = 0;
index = 0;
while(j < len(data)):
    j += day_per_mouth[index]
    if(j > len(data)):
      j = len(data)
    temp_data = data[i:j]
    for col_name in temp_data.columns:
      temp_data =  temp_data.fillna(temp_data.mean())
      temp_data = filling_out(temp_data)
    index += 1;
    if(index > 11):
        index = 0
    i = j

In [150]:
data = data.fillna(data.mean())
data = filling_out(data);
print(data.isnull().sum())

Date             0
Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
RainToday        0
RISK_MM          0
RainTomorrow     0
dtype: int64


In [151]:
data.to_csv('finalv1.1.csv')

In [152]:
X = data.drop('RainTomorrow', axis=1) # assuming the target variable is named 'target'
y = data['RainTomorrow']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [153]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model on the training data
rf.fit(X_train, y_train)

# Evaluate the model on the testing data
score = rf.score(X_test, y_test)
print("Accuracy:", score)

Accuracy: 1.0
